In [1]:
import pandas as pd
import scipy.stats as st
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
import math
import traceback
%matplotlib inline

In [2]:
# load data
df = pd.read_csv('health care diabetes.csv')
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [3]:
df_linear = pd.read_csv('Salary_dataset.csv')
df_linear

,Unnamed: 0,YearsExperience,Salary
0,0,1.2,39344.0
1,1,1.4,46206.0
2,2,1.6,37732.0
3,3,2.1,43526.0
4,4,2.3,39892.0
5,5,3.0,56643.0
6,6,3.1,60151.0
7,7,3.3,54446.0
8,8,3.3,64446.0
9,9,3.8,57190.0


In [4]:
# global variables
X = df.drop(columns=['Outcome']) # all input values
y = df['Outcome']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
print(f"X train: {X_train} Type: {type(X_train)}\n") # 2D array of np.float64
print(f"X test: {X_test} Type: {type(X_test)}\n")
print(f"y train: {y_train} Type: {type(y_train)}\n") # pd.Series array of ints
print(f"y test: {y_test} Type: {type(y_test)}\n")

model = LogisticRegression()
model.fit(X_train, y_train)
print(f"Accuracy: {model.score(X_test, y_test)}")

X train:      Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
287            1      119             86             39      220  45.6   
78             0      131              0              0        0  43.2   
492            4       99             68             38        0  32.8   
337            5      115             76              0        0  31.2   
457            5       86             68             28       71  30.2   
..           ...      ...            ...            ...      ...   ...   
94             2      142             82             18       64  24.7   
35             4      103             60             33      192  24.0   
458           10      148             84             48      237  37.6   
74             1       79             75             30        0  32.0   
235            4      171             72              0        0  43.6   

     DiabetesPedigreeFunction  Age  
287                     0.808   29  
78                      0.27

C:\Users\ezrab\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [6]:
model.predict(X_test)

array([0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [7]:
# global variables
clusters = [] # stores x values of data
m_init = 1
b_init = 0
learning_rate = 0.01 # default
points = []
iterations = 100
prev_val = 0
epsilon = 0.001 # convergence test number, if y changes by a value less than ε then declare convergence
weights = [1, 1, 1, 1, 1, 1, 1, 1] # 8 predictor variable weights
# h(x) = w1x1 + w2x2 + w3x3 + ... + w(n)x(n)

In [86]:
# supplemental functions

def sigmoid_function(x, m, b): # x is a vector of data
    y_temp_pred = b
    i = 0
    for val in x: # length of x == i
        y_temp_pred += val * weights[i]
        
    return 1/(1 + math.exp(-1*y_temp_pred))

def linear_regression(x, m, b):
    y_temp_pred = b
    i = 0
    for val in x: # length of x == i
        y_temp_pred += val * weights[i]
    print(y_temp_pred)
              
# optimize model using stochastic gradient descent

def cluster_points():
    # increment by some x and use a circle with radius x/2 to capture points
    print("Clustering points")
    increment = 10 # diameter of circle
    start = min(X_adjusted)
    end = max(X_adjusted)
    # (x - h)^2 + (y - k)^2 <= r^2
    radius = increment/2
    X_center = increment/2
    y_center = sigmoid_function(X_center, m_init, b_init)
    
    for i in range(start, end, increment):
        temp = []
        for j in range(start, end):
            if ((X_adjusted[j] - X_center)**2 + (y_adjusted[j] - y_center)**2) <= (radius)**2:
                temp.append(X_adjusted[j])
            
        if len(temp) > 0:
            clusters.append(temp)
        
        X_center = X_center + increment
        y_center = sigmoid_function(X_center, m_init, b_init)
        print(f"{X_center}, {y_center}")
        
        remove_duplicates()
    print(clusters)
    
def remove_duplicates():
    temp = dict()
    for i in range(0, len(clusters)):
        for j in range(0, len(clusters[i])):
            if clusters[i][j] not in temp.keys():
                temp[clusters[i][j]] = 1
            else:
                clusters[i].pop(j)
        
    
def random_cluster_points(): # empty range for randrange() (0,0,0)
    print("Selecting a random point per cluster")
    for cluster in clusters:
        if len(cluster) >= 1:
            if len(cluster) > 1:
                i = random.randint(0, len(cluster)-1)
                points.append(cluster[i])
            else:
                points.append(cluster[0])

    print(points)
    

# partial derivative of loss function with respect to slope
def derivative_m(x, y, m, b):
    return -2*x*(y[0]-(m*x+b)/len(weights))
    
    
# partial derivative of loss function with respect to y-intercept
def derivative_b(x, y, m, b):
    return -2*(y[0]-(m*x+b)/len(weights))


def rms_error(predicted, actual):
    err = 0
    err = (predicted - actual)**2
    mean_squared_err = math.sqrt(err/y_train.size)
    
    print(f"N = {y_train.size}")
    print(f"∑(x-xi)² = {err}")
    
    return mean_squared_err
    
# def fit_model():
    

def stochastic_gradient_descent(m, b):
    print("Performing SGD")
    sample = []
    
    for i in range(0, iterations):
        print(f"Iteration {i+1}")
        global m_init
        global b_init
        global prev_val
                
        rand_index = np.random.randint(0, X_train['Pregnancies'].size) # Used Pregnancies as placeholder category for # rows

        rand_val = X_train.iloc[rand_index]
        print(rand_val)
        sample.append(rand_val)
        actual = model.predict(sample)
        # need to work on my own fit and prediction
        curr_val = sigmoid_function(rand_val, m_init, b_init)
        print(f"Current value: {curr_val} | i: {i}\n | actual: {actual}")
        
        adjusted_m = derivative_m(curr_val, actual, m_init, b_init) # error here
        adjusted_b = derivative_b(curr_val, actual, m_init, b_init)
        print(f"Previous value: {prev_val} | Actual: {actual}\n")
        sample.clear()
        
        if (abs(curr_val - prev_val) < epsilon and i > iterations/2):
            print('Model converged!')
            break

        m_init = m_init - adjusted_m * learning_rate
        b_init = b_init - adjusted_b * learning_rate
        print(f"New slope: {m_init}\n") # not working
        print(f"New intercept: {b_init}\n")
        prev_val = curr_val

In [87]:
stochastic_gradient_descent(m_init, b_init)

Performing SGD
Iteration 1
Pregnancies                   7.000
Glucose                     107.000
BloodPressure                74.000
SkinThickness                 0.000
Insulin                       0.000
BMI                          29.600
DiabetesPedigreeFunction      0.254
Age                          31.000
Name: 17, dtype: float64
Current value: 1.0 | i: 0
 | actual: [0]
Previous value: 1.0 | Actual: [0]

New slope: 0.9975

New intercept: -0.0025

Iteration 2
Pregnancies                   1.000
Glucose                     143.000
BloodPressure                86.000
SkinThickness                30.000
Insulin                     330.000
BMI                          30.100
DiabetesPedigreeFunction      0.892
Age                          23.000
Name: 574, dtype: float64
Current value: 1.0 | i: 1
 | actual: [0]
Previous value: 1.0 | Actual: [0]

New slope: 0.9950125000000001

New intercept: -0.004987500000000001

Iteration 3
Pregnancies                   1.000
Glucose               

In [46]:
# Tasks for now:
# logistic regression sklearn, compare my rms to built in ^
# ensure model has converged
# calculate root mean square error for each point ^
# convergence - rms error converges to smallest value it could achieve 

# print(f"My Mean squared error formula: {rms_error()}")
print(f"Sklearn Mean squared error: {mean_squared_error(model.predict(X_test), y_test)}")

Sklearn Mean squared error: 0.22395833333333334
